In [1]:
import pandas as pd
import csv 

path = "../cv-cat-18/ca/validated.tsv"

In [147]:
df_full = pd.read_csv(path,delimiter="\t", escapechar="\\",quoting = csv.QUOTE_NONE)

/var/folders/2d/7d3c_x1x5mb40mc1sgr14p740000gn/T/ipykernel_24625/3946300290.py:1: DtypeWarning: Columns (4,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv(path,delimiter="\t", escapechar="\\",quoting = csv.QUOTE_NONE)


In [148]:
import json

with open("./macrodial.json",'r',encoding='utf-8') as input: 
    macro_dial = json.load(input)

In [149]:
df_full["grouped_accents"] = df_full.apply(lambda row: macro_dial[row.accents] if macro_dial.get(row.accents, None) else "None",axis=1)

In [150]:
len(df_full)

1892545

In [151]:
#Remove repeats that occur in same macro-dialect
df_full = df_full.drop_duplicates(subset=["sentence","grouped_accents"])
len(df_full)

1578278

In [152]:
df_val = df_full[df_full.grouped_accents=='valencià']
df_cent = df_full[df_full.grouped_accents=='central']
df_nord = df_full[df_full.grouped_accents=='nord']
df_bal = df_full[df_full.grouped_accents=='balear']
df_no = df_full[df_full.grouped_accents=='nord-occidental']

In [153]:
val_eval = df_val.sample(n=2486)
cent_eval = df_cent.sample(n=2486)
nord_eval = df_nord.sample(n=2486)
bal_eval = df_bal.sample(n=2486)
no_eval = df_no.sample(n=2486)


In [154]:
df_full.columns

Index(['client_id', 'path', 'sentence_id', 'sentence', 'sentence_domain',
       'up_votes', 'down_votes', 'age', 'gender', 'accents', 'variant',
       'locale', 'segment', 'grouped_accents'],
      dtype='object')

In [155]:
full_eval = pd.concat([val_eval,cent_eval,nord_eval,bal_eval,no_eval],ignore_index=True)

In [156]:
out_path = "../cv-cat-18/ca/"
full_eval.to_csv(out_path+"eval_balanced.tsv",sep='\t',index=False)

In [157]:
len(df_full)

1578278

In [158]:
train_val_df = df_full.drop(pd.concat([val_eval,cent_eval,nord_eval,bal_eval,no_eval]).index)

In [159]:
len(train_val_df)

1565848

In [160]:
#Check that the lengths of the train_val set correspond to what they should after removing eval
assert (len(df_full)-len(train_val_df)) == len(full_eval)

In [161]:
pd.concat([val_eval,cent_eval,nord_eval,bal_eval,no_eval]).head()

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment,grouped_accents
376288,1d54d194cff4675f590e0e73e98f6392358891d09dd19b...,common_voice_ca_19031942.mp3,0f935e0f31a3462e7d934c568af5054def5f87595f7879...,La mort.,NaN,3,0,fifties,male_masculine,valencià,NaN,ca,NaN,valencià
749002,e7847a5814b865bc043600fee7d810b9815da389278fdf...,common_voice_ca_23910869.mp3,be7816b979462943ae67b7459b5356e3f3d577210a2ab4...,"Ocasionalment reaparegué el seu nom, més, enca...",NaN,2,0,teens,male_masculine,valencià,NaN,ca,NaN,valencià
1048366,dafd89491990553f5e22021f96344b3bc92be6a419c919...,common_voice_ca_20131812.mp3,1d1b58b48145e454ded466211cb4d29844b1f2b4eecbf9...,Això va permetre la creació dels primers canal...,NaN,3,0,twenties,male_masculine,valencià,NaN,ca,NaN,valencià
1529614,892bf89bd3a008a7d982de0d278349e654c713efacf965...,common_voice_ca_38406447.mp3,aff8538286761070459fa83f159bd919d98b2390e973c6...,Actualment és un mineral aprovat per l'Associa...,NaN,3,0,fourties,male_masculine,"valencià,La Vall d'Albaida",NaN,ca,NaN,valencià
730340,7d19dccf48114d3ec00c45fe80581300faca042157d6c9...,common_voice_ca_20757891.mp3,358cefe95803b4cb70ff108b545363767d944e17e6ee4d...,"S'utilitza sobre animacions publicitàries, cur...",NaN,3,0,thirties,male_masculine,valencià,NaN,ca,NaN,valencià


In [162]:
print(376288 in list(train_val_df.index))

False


In [163]:
print(730340 in list(train_val_df.index))

False


In [164]:
train_val_df.to_csv(out_path+"train_dev_full.tsv",sep='\t',index=False)

# First train/dev split: 
* 100% Central Catalan

In [165]:
train_dev_val_full = train_val_df[train_val_df.grouped_accents=='valencià']
train_dev_cen_full = train_val_df[train_val_df.grouped_accents=='central']
train_dev_nord_full = train_val_df[train_val_df.grouped_accents=='nord']
train_dev_bal_full = train_val_df[train_val_df.grouped_accents=='balear']
train_dev_no_full = train_val_df[train_val_df.grouped_accents=='nord-occidental']

In [166]:
len(train_dev_cen_full)

758531

In [167]:
# for this split, train has 99455 (central) samples; dev has 12430 samples

central_dev_100 = train_dev_cen_full.sample(n=12430)
temp = train_dev_cen_full.drop(central_dev_100.index)

In [168]:
len(temp)

746101

In [169]:
central_train_100 = temp.sample(n=99455)

In [170]:
#Save both these 
central_dev_100.to_csv(out_path+"dev_100.tsv",sep='\t',index=False)
central_train_100.to_csv(out_path+"train_100.tsv",sep='\t',index=False)

In [171]:
len(central_dev_100)

12430

# Second Train/dev split:
* 80% Central (79564 train samples, 9944 dev samples)
* 5% each of Valencià, nord, nord-occidental, and balear (4972 train samples each, 621 dev samples each)

In [172]:
#Central:
central_dev_80 = train_dev_cen_full.sample(n=9944)
temp = train_dev_cen_full.drop(central_dev_80.index)
central_train_80 = temp.sample(n=79564)

In [173]:
#Val
val_dev_80 = train_dev_val_full.sample(n=621)
temp = train_dev_val_full.drop(val_dev_80.index)
val_train_80 = temp.sample(n=4972)

In [174]:
#nord
nord_dev_80 = train_dev_nord_full.sample(n=621)
temp = train_dev_nord_full.drop(nord_dev_80.index)
nord_train_80 = temp.sample(n=4972)

In [175]:
#nord-occidental
no_dev_80 = train_dev_no_full.sample(n=621)
temp = train_dev_no_full.drop(no_dev_80.index)
no_train_80 = temp.sample(n=4972)

In [176]:
#Balear
bal_dev_80 = train_dev_bal_full.sample(n=621)
temp = train_dev_bal_full.drop(bal_dev_80.index)
bal_train_80 = temp.sample(n=4972)

In [177]:
#Concatenate everything and save
full_train_80 = pd.concat([central_train_80,val_train_80,nord_train_80,no_train_80,bal_train_80],ignore_index=True)
full_dev_80 = pd.concat([central_dev_80,val_dev_80,nord_dev_80,no_dev_80,bal_dev_80],ignore_index=True)

In [178]:
len(full_train_80[full_train_80.grouped_accents=="nord"])

4972

In [179]:
#Save
full_dev_80.to_csv(out_path+"dev_80.tsv",sep='\t',index=False)
full_train_80.to_csv(out_path+"train_80.tsv",sep='\t',index=False)

# Third Train/dev split:
* 50% Central (49727 train samples, 6215 dev samples)
* 12.5% each of Valencià, nord, nord-occidental, and balear (12431 train samples each, 1553 dev samples each)

In [180]:
#Central:
central_dev_50 = train_dev_cen_full.sample(n=6215)
temp = train_dev_cen_full.drop(central_dev_50.index)
central_train_50 = temp.sample(n=49727)

In [181]:
#Val
val_dev_50 = train_dev_val_full.sample(n=1553)
temp = train_dev_val_full.drop(val_dev_50.index)
val_train_50 = temp.sample(n=12431)

In [182]:
#nord
nord_dev_50 = train_dev_nord_full.sample(n=1553)
temp = train_dev_nord_full.drop(nord_dev_50.index)
nord_train_50 = temp.sample(n=12431)

In [183]:
#nord-occidental
no_dev_50 = train_dev_no_full.sample(n=1553)
temp = train_dev_no_full.drop(no_dev_50.index)
no_train_50 = temp.sample(n=12431)

In [184]:
#Balear
bal_dev_50 = train_dev_bal_full.sample(n=1553)
temp = train_dev_bal_full.drop(bal_dev_50.index)
bal_train_50 = temp.sample(n=12431)

In [185]:
#Concatenate everything and save
full_train_50 = pd.concat([central_train_50,val_train_50,nord_train_50,no_train_50,bal_train_50],ignore_index=True)
full_dev_50 = pd.concat([central_dev_50,val_dev_50,nord_dev_50,no_dev_50,bal_dev_50],ignore_index=True)

In [186]:
len(full_train_50[full_train_50.grouped_accents=="balear"])

12431

In [187]:
#Save
full_dev_50.to_csv(out_path+"dev_50.tsv",sep='\t',index=False)
full_train_50.to_csv(out_path+"train_50.tsv",sep='\t',index=False)

# Final Train/dev split:
* All dialects balanced: 2486 dev samples of each, 19891 train samples of each

In [188]:
#Central:
central_dev_20 = train_dev_cen_full.sample(n=2486)
temp = train_dev_cen_full.drop(central_dev_20.index)
central_train_20 = temp.sample(n=19891)

In [189]:
#Val
val_dev_20 = train_dev_val_full.sample(n=2486)
temp = train_dev_val_full.drop(val_dev_20.index)
val_train_20 = temp.sample(n=19891)

In [190]:
#nord
nord_dev_20 = train_dev_nord_full.sample(n=2486)
temp = train_dev_nord_full.drop(nord_dev_20.index)
nord_train_20 = temp.sample(n=19891)

In [191]:
#nord-occidental
no_dev_20 = train_dev_no_full.sample(n=2486)
temp = train_dev_no_full.drop(no_dev_20.index)
no_train_20 = temp.sample(n=19891)

In [192]:
#Balear
bal_dev_20 = train_dev_bal_full.sample(n=2486)
temp = train_dev_bal_full.drop(bal_dev_20.index)
bal_train_20 = temp.sample(n=19891)

In [193]:
#Concatenate everything and save
full_train_20 = pd.concat([central_train_20,val_train_20,nord_train_20,no_train_20,bal_train_20],ignore_index=True)
full_dev_20 = pd.concat([central_dev_20,val_dev_20,nord_dev_20,no_dev_20,bal_dev_20],ignore_index=True)

In [194]:
len(full_train_20[full_train_20.grouped_accents=="central"])

19891

In [195]:
#Save
full_dev_20.to_csv(out_path+"dev_20.tsv",sep='\t',index=False)
full_train_20.to_csv(out_path+"train_20.tsv",sep='\t',index=False)

# Split the Adversarial samples off the eval set. 

In [2]:
eval = pd.read_csv("../eval_balanced.tsv",delimiter="\t", escapechar="\\",quoting = csv.QUOTE_NONE)

In [3]:
#Check there are the correct amount of samples (12430)
len(eval)

12430

In [4]:
val_eval = eval[eval.grouped_accents=='valencià']
central_eval = eval[eval.grouped_accents=='central']
nord_eval=eval[eval.grouped_accents=='nord']
bal_eval = eval[eval.grouped_accents=='balear']
no_eval = eval[eval.grouped_accents=='nord-occidental']


In [5]:
val_cw = val_eval.sample(n=50)
central_cw = central_eval.sample(n=50)
nord_cw = nord_eval.sample(n=50)
bal_cw = bal_eval.sample(n=50)
no_cw = no_eval.sample(n=50)

In [6]:
#Concatenate everything and save
full_cw_set = pd.concat([val_cw,central_cw,nord_cw,bal_cw,no_cw],ignore_index=True)
len(full_cw_set)

250

In [7]:
#Save
full_cw_set.to_csv("../attack_samp_balanced.tsv",sep='\t',index=False)